In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import sys, importlib
PROJECT_ROOT = "/content/drive/MyDrive/SA_CropType_SourceCoop"
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

import common
importlib.reload(common)
from common import *

ensure_dirs()

print("✅ BASE_DIR:", BASE_DIR)
print("✅ FIELD_INFO_TRAIN:", FIELD_INFO_TRAIN, "exists:", FIELD_INFO_TRAIN.exists())
print("✅ LABELS_DIR:", LABELS_DIR, "exists:", LABELS_DIR.exists())
print("✅ S2_DIR:", S2_DIR, "exists:", S2_DIR.exists())
print("✅ S1_DIR:", S1_DIR, "exists:", S1_DIR.exists())
print("✅ OUT_DIR:", OUT_DIR, "exists:", OUT_DIR.exists())
print("✅ TFC_FILE:", TFC_FILE)

Mounted at /content/drive
✅ BASE_DIR: /content/drive/MyDrive/SA_CropType_SourceCoop
✅ FIELD_INFO_TRAIN: /content/drive/MyDrive/SA_CropType_SourceCoop/field_info_train.csv exists: True
✅ LABELS_DIR: /content/drive/MyDrive/SA_CropType_SourceCoop/train/labels exists: True
✅ S2_DIR: /content/drive/MyDrive/SA_CropType_SourceCoop/train/imagery/s2 exists: True
✅ S1_DIR: /content/drive/MyDrive/SA_CropType_SourceCoop/train/imagery/s1 exists: True
✅ OUT_DIR: /content/drive/MyDrive/SA_CropType_SourceCoop/outputs exists: True
✅ TFC_FILE: /content/drive/MyDrive/SA_CropType_SourceCoop/outputs/meta/tile_field_crop_table.csv


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import os, json
from tqdm import tqdm

In [ ]:
# ============================================================
# ORTAK YARDIMCI FONKSİYONLAR
# ============================================================

def normalize_columns(df):
    """
    CSV kolon adlarını standart hale getirir:
    - Boşlukları temizler
    - 'Field ID' -> 'field_id'
    - 'Crop' -> 'crop'
    """
    # boşlukları temizle
    df.columns = [c.strip() for c in df.columns]

    # kolon isimlerini normalize et
    rename_map = {
        "Field ID": "field_id",
        "Field_Id": "field_id",
        "FIELD_ID": "field_id",
        "Crop": "crop",
        "CROP": "crop"
    }

    df = df.rename(columns=rename_map)
    return df

In [ ]:
import pandas as pd

df = pd.read_csv(FIELD_INFO_TRAIN)
df = normalize_columns(df)

need = ["field_id", "crop"]
missing = [c for c in need if c not in df.columns]
if missing:
    raise ValueError(f"Eksik kolonlar: {missing}. Mevcut kolonlar: {df.columns.tolist()}")

# field_id sayısala çevir
df["field_id"] = pd.to_numeric(df["field_id"], errors="coerce")
df = df.dropna(subset=["field_id"]).copy()
df["field_id"] = df["field_id"].astype(int)

print("✅ df kolonları:", df.columns.tolist())
print("rows:", len(df), "| unique fields:", df["field_id"].nunique(), "| classes:", df["crop"].nunique())
df.head()

✅ df kolonları: ['field_id', 'crop']
rows: 87113 | unique fields: 87113 | classes: 9


,field_id,crop
0,84337,Fallow
1,91979,Weeds
2,119964,Weeds
3,43134,Weeds
4,95735,Fallow


In [ ]:
import numpy as np
import rasterio
from tqdm import tqdm

# field_id -> crop sözlüğü
lookup = df.drop_duplicates("field_id").set_index("field_id")["crop"].to_dict()

label_files = sorted(LABELS_DIR.glob("*_field_ids.tif"))
print("Label tif sayısı:", len(label_files), "| örnek:", [p.name for p in label_files[:5]])

rows = []
for p in tqdm(label_files):
    tile_id = int(p.stem.split("_")[0])  # "100_field_ids" -> 100
    with rasterio.open(p) as src:
        arr = src.read(1)

    fids = np.unique(arr)
    fids = fids[fids > 0]  # 0 = NoData

    for fid in fids:
        rows.append((tile_id, int(fid), lookup.get(int(fid), "UNK")))

tfc = pd.DataFrame(rows, columns=["tile_id", "field_id", "crop"])
print("tfc:", tfc.shape, "| unique tiles:", tfc["tile_id"].nunique(), "| unique fields:", tfc["field_id"].nunique())
tfc.head()

Label tif sayısı: 499 | örnek: ['1000_field_ids.tif', '1001_field_ids.tif', '1002_field_ids.tif', '1003_field_ids.tif', '1004_field_ids.tif']


100%|██████████| 499/499 [00:36<00:00, 13.63it/s]


tfc: (16893, 3) | unique tiles: 499 | unique fields: 16893


,tile_id,field_id,crop
0,1000,10697,Weeds
1,1000,14604,Lucerne/Medics
2,1000,17138,Fallow
3,1000,22246,Lucerne/Medics
4,1000,25176,Fallow


In [ ]:
out_path = META_DIR / "tile_field_crop_table.csv"
tfc.to_csv(out_path, index=False)
print("✅ Kaydedildi:", out_path)

✅ Kaydedildi: /content/drive/MyDrive/SA_CropType_SourceCoop/outputs/meta/tile_field_crop_table.csv
